# 03 - Model Training

Notebook ini berisi training LSTM neural network untuk prediksi harga saham.

## Steps:
1. Load preprocessed data
2. Build LSTM architecture
3. Train model dengan callbacks
4. Evaluate model performance
5. Visualize predictions vs actual

In [ ]:
# Import libraries
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# TensorFlow
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

# Check GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPU available: {gpus}")
else:
    print("No GPU found. Training will use CPU.")

# Import custom modules
from src.model import LSTMStockPredictor, StackedLSTMPredictor
from src.preprocessor import StockPreprocessor
from src.visualizer import StockVisualizer
from src.data_loader import DataLoader

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("\nLibraries imported successfully!")

## 1. Load Preprocessed Data

In [ ]:
# Load preprocessed data
X_train = np.load('../data/processed/X_train.npy')
y_train = np.load('../data/processed/y_train.npy')
X_test = np.load('../data/processed/X_test.npy')
y_test = np.load('../data/processed/y_test.npy')

print("=== Data Loaded ===")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

In [ ]:
# Load scaler for inverse transform
preprocessor = StockPreprocessor()
preprocessor.load_scaler('../models/scaler.pkl')

# Get sequence length and features from data shape
SEQUENCE_LENGTH = X_train.shape[1]
N_FEATURES = X_train.shape[2]

print(f"\nSequence Length: {SEQUENCE_LENGTH}")
print(f"Number of Features: {N_FEATURES}")

## 2. Model Configuration

In [ ]:
# Model hyperparameters
config = {
    'sequence_length': SEQUENCE_LENGTH,
    'n_features': N_FEATURES,
    'lstm_units': [50, 50, 50],  # 3 LSTM layers
    'dropout_rate': 0.2,
    'learning_rate': 0.001,
    
    # Training params
    'epochs': 100,
    'batch_size': 32,
    'patience': 10,  # Early stopping patience
}

print("=== Model Configuration ===")
for key, value in config.items():
    print(f"{key}: {value}")

## 3. Build LSTM Model

In [ ]:
# Initialize model
model = LSTMStockPredictor(
    sequence_length=config['sequence_length'],
    n_features=config['n_features'],
    lstm_units=config['lstm_units'],
    dropout_rate=config['dropout_rate'],
    learning_rate=config['learning_rate']
)

# Build model architecture
model.build_model()

In [ ]:
# Visualize model architecture
tf.keras.utils.plot_model(
    model.model, 
    to_file='../models/model_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    dpi=150
)

# Display the image
from IPython.display import Image
Image('../models/model_architecture.png')

## 4. Train Model

In [ ]:
# Train model
history = model.train(
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    epochs=config['epochs'],
    batch_size=config['batch_size'],
    model_path='../models/lstm_model.keras',
    patience=config['patience']
)

In [ ]:
# Plot training history
model.plot_training_history(save_path='../models/training_history.png')

## 5. Model Evaluation

In [ ]:
# Evaluate on test set
metrics = model.evaluate(X_test, y_test)

In [ ]:
# Make predictions
predictions_scaled = model.predict(X_test)

# Inverse transform to original scale
predictions = preprocessor.inverse_transform_predictions(predictions_scaled)
actual = preprocessor.inverse_transform_actual(y_test)

print(f"\nPredictions shape: {predictions.shape}")
print(f"Actual shape: {actual.shape}")

# Sample predictions
print("\nSample Predictions vs Actual:")
for i in range(5):
    print(f"  Actual: ${actual[i][0]:.2f}, Predicted: ${predictions[i][0]:.2f}")

## 6. Visualization

In [ ]:
# Initialize visualizer
viz = StockVisualizer()

# Plot actual vs predicted
viz.plot_predictions(
    actual=actual.flatten(),
    predicted=predictions.flatten(),
    title='LSTM Stock Price Prediction - Actual vs Predicted',
    save_path='../models/predictions.png'
)

In [ ]:
# Get training data for full visualization
loader = DataLoader(data_dir='../data/raw')
TICKER = "AAPL"

try:
    full_data = loader.load_data(TICKER)
except:
    full_data = loader.download_stock_data(TICKER, period="5y")

# Get training target values
train_size = len(y_train)
train_prices = full_data['Close'].values[SEQUENCE_LENGTH:SEQUENCE_LENGTH + train_size]

# Plot with training data
viz.plot_train_test_split(
    train_data=train_prices,
    test_actual=actual.flatten(),
    test_predicted=predictions.flatten(),
    title=f'{TICKER} LSTM Prediction - Training & Test Data',
    save_path='../models/full_prediction.png'
)

In [ ]:
# Error analysis
viz.plot_prediction_error(
    actual=actual.flatten(),
    predicted=predictions.flatten(),
    title='Prediction Error Analysis',
    save_path='../models/error_analysis.png'
)

In [ ]:
# Create comprehensive report
viz.create_summary_report(
    data=full_data,
    actual=actual.flatten(),
    predicted=predictions.flatten(),
    metrics=metrics,
    ticker=TICKER,
    save_dir='../models'
)

## 7. Future Price Prediction

In [ ]:
# Predict next day price
last_sequence = preprocessor.prepare_prediction_data(full_data)
next_day_scaled = model.predict(last_sequence)
next_day_price = preprocessor.inverse_transform_predictions(next_day_scaled)

current_price = full_data['Close'].iloc[-1]
predicted_change = (next_day_price[0][0] - current_price) / current_price * 100

print("\n" + "="*50)
print("NEXT DAY PREDICTION")
print("="*50)
print(f"Current Price: ${current_price:.2f}")
print(f"Predicted Next Day: ${next_day_price[0][0]:.2f}")
print(f"Expected Change: {predicted_change:+.2f}%")
print("="*50)

if predicted_change > 0:
    print("\n📈 Model predicts UPWARD movement")
else:
    print("\n📉 Model predicts DOWNWARD movement")

## 8. Save Final Model

In [ ]:
# Save final model
model.save_model('../models/lstm_final_model.keras')

# Save metrics to JSON
import json

metrics_to_save = {
    'ticker': TICKER,
    'sequence_length': SEQUENCE_LENGTH,
    'n_features': N_FEATURES,
    'loss': float(metrics['loss']),
    'mae': float(metrics['mae']),
    'rmse': float(metrics['rmse']),
    'mape': float(metrics['mape']),
    'train_samples': len(y_train),
    'test_samples': len(y_test),
    'config': config
}

with open('../models/model_metrics.json', 'w') as f:
    json.dump(metrics_to_save, f, indent=2)

print("Model and metrics saved!")

## 9. Summary

### Model Training Complete! ✅

| Metric | Value |
|--------|-------|
| Loss (MSE) | See above |
| MAE | See above |
| RMSE | See above |
| MAPE | See above |

### Files Saved:
- `models/lstm_final_model.keras` - Trained model
- `models/scaler.pkl` - Data scaler
- `models/model_metrics.json` - Evaluation metrics
- `models/training_history.png` - Training curves
- `models/predictions.png` - Prediction plots
- `models/error_analysis.png` - Error analysis

### Notes:
- Model menggunakan 3 LSTM layers dengan dropout untuk mencegah overfitting
- Early stopping digunakan untuk menghentikan training saat model mulai overfit
- Learning rate scheduler mengurangi learning rate saat loss tidak membaik

### ⚠️ Disclaimer:
This is for educational purposes only. Stock price prediction is inherently uncertain and this model should NOT be used for actual trading decisions.

In [ ]:
print("\n" + "="*60)
print("🎉 LSTM STOCK PREDICTION MODEL TRAINING COMPLETE!")
print("="*60)
print(f"\nTicker: {TICKER}")
print(f"Model: LSTM with {len(config['lstm_units'])} layers")
print(f"Best Validation Loss: {min(history['val_loss']):.6f}")
print(f"\nModel saved to: ../models/lstm_final_model.keras")
print("\n⚠️  Remember: Past performance does not guarantee future results!")